In [ ]:
import xarray as xr
from rasterstats import zonal_stats
import geopandas as gpd
from shapely.geometry import Point
import glob

In [ ]:
ug = gpd.read_file(
    "C:/Users/Administrator/OneDrive/Desktop/rainfall_prediction/uganda_administrative_boundaries/simplified_uganda_districts.shp"
)

ug.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [18]:
ug.head()

,ADM2_EN,ADM1_EN,geometry
0,Abim,Northern,"POLYGON ((33.58099 3.14912, 33.58555 3.14702, ..."
1,Adjumani,Northern,"POLYGON ((32.04597 3.58896, 32.04578 3.58875, ..."
2,Agago,Northern,"POLYGON ((33.41652 3.30434, 33.41701 3.30403, ..."
3,Alebtong,Northern,"POLYGON ((33.03629 2.5013, 33.03664 2.50075, 3..."
4,Amolatar,Northern,"POLYGON ((32.9014 1.8057, 32.90231 1.80568, 32..."


In [ ]:
ds = xr.open_dataset(
    "C:/Users/Administrator/OneDrive/Desktop/rainfall_prediction/era5/dewpoint_temperature/humidity1981_1993.grib",
    engine="cfgrib",
)

ds

<xarray.Dataset> Size: 3GB
Dimensions:     (time: 113952, latitude: 24, longitude: 259)
Coordinates:
  * time        (time) datetime64[ns] 912kB 1981-01-01 ... 1993-12-31T23:00:00
  * latitude    (latitude) float64 192B 4.25 4.0 3.75 3.5 ... -1.0 -1.25 -1.5
  * longitude   (longitude) float64 2kB -29.5 -29.25 -29.0 ... 34.5 34.75 35.0
    number      int64 8B ...
    step        timedelta64[ns] 8B ...
    surface     float64 8B ...
    valid_time  (time) datetime64[ns] 912kB ...
Data variables:
    d2m         (time, latitude, longitude) float32 3GB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-20T02:34 GRIB to CDM+CF via cfgrib-0.9.1...

In [20]:
minx, miny, maxx, maxy = ug.total_bounds

ds_ug = ds.sel(longitude=slice(minx, maxx), latitude=slice(maxy, miny))
ds_ug

<xarray.Dataset> Size: 222MB
Dimensions:     (time: 113952, latitude: 22, longitude: 22)
Coordinates:
  * time        (time) datetime64[ns] 912kB 1981-01-01 ... 1993-12-31T23:00:00
  * latitude    (latitude) float64 176B 4.0 3.75 3.5 3.25 ... -0.75 -1.0 -1.25
  * longitude   (longitude) float64 176B 29.75 30.0 30.25 ... 34.5 34.75 35.0
    number      int64 8B ...
    step        timedelta64[ns] 8B ...
    surface     float64 8B ...
    valid_time  (time) datetime64[ns] 912kB ...
Data variables:
    d2m         (time, latitude, longitude) float32 221MB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-20T02:34 GRIB to CDM+CF via cfgrib-0.9.1...

In [ ]:
ds_monthly = ds_ug.resample(time="1M").mean()
ds_monthly

c:\Program Files\Python311\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(


<xarray.Dataset> Size: 304kB
Dimensions:    (time: 156, latitude: 22, longitude: 22)
Coordinates:
  * time       (time) datetime64[ns] 1kB 1981-01-31 1981-02-28 ... 1993-12-31
  * latitude   (latitude) float64 176B 4.0 3.75 3.5 3.25 ... -0.75 -1.0 -1.25
  * longitude  (longitude) float64 176B 29.75 30.0 30.25 ... 34.5 34.75 35.0
    number     int64 8B 0
    step       timedelta64[ns] 8B 00:00:00
    surface    float64 8B 0.0
Data variables:
    d2m        (time, latitude, longitude) float32 302kB 287.9 287.2 ... 284.8
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-20T02:34 GRIB to CDM+CF via cfgrib-0.9.1...

In [ ]:
files = glob.glob(
    "C:/Users/Administrator/OneDrive/Desktop/rainfall_prediction/era5/dewpoint_temperature/*.grib"
)

datasets = []

for file in files:
    ds = xr.open_dataset(file, engine="cfgrib")
    minx, miny, maxx, maxy = ug.total_bounds
    ds_ug = ds.sel(longitude=slice(minx, maxx), latitude=slice(maxy, miny))
    ds_monthly = ds_ug.resample(time="1M").mean()
    datasets.append(ds_monthly)

combined_ds = xr.concat(datasets, dim="time")

combined_ds

Ignoring index file 'C:/Users/Administrator/OneDrive/Desktop/rainfall_prediction/era5/dewpoint_temperature\\humidity1981_1993.grib.5b7b6.idx' incompatible with GRIB file
c:\Program Files\Python311\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(
c:\Program Files\Python311\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(
c:\Program Files\Python311\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(
c:\Program Files\Python311\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(


<xarray.Dataset> Size: 1MB
Dimensions:    (time: 539, latitude: 22, longitude: 22)
Coordinates:
  * time       (time) datetime64[ns] 4kB 1981-01-31 1981-02-28 ... 2025-11-30
  * latitude   (latitude) float64 176B 4.0 3.75 3.5 3.25 ... -0.75 -1.0 -1.25
  * longitude  (longitude) float64 176B 29.75 30.0 30.25 ... 34.5 34.75 35.0
    number     int64 8B 0
    step       timedelta64[ns] 8B 00:00:00
    surface    float64 8B 0.0
Data variables:
    d2m        (time, latitude, longitude) float32 1MB 287.9 287.2 ... 285.9
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-20T02:39 GRIB to CDM+CF via cfgrib-0.9.1...

In [ ]:
df_dewpoint = combined_ds.to_dataframe().reset_index()

df_dewpoint

,time,latitude,longitude,d2m,number,step,surface
0,1981-01-31,4.00,29.75,287.893951,0,0 days,0.0
1,1981-01-31,4.00,30.00,287.198517,0,0 days,0.0
2,1981-01-31,4.00,30.25,286.702942,0,0 days,0.0
3,1981-01-31,4.00,30.50,285.920441,0,0 days,0.0
4,1981-01-31,4.00,30.75,285.420624,0,0 days,0.0
...,...,...,...,...,...,...,...
260871,2025-11-30,-1.25,34.00,291.429352,0,0 days,0.0
260872,2025-11-30,-1.25,34.25,289.202240,0,0 days,0.0
260873,2025-11-30,-1.25,34.50,287.637360,0,0 days,0.0
260874,2025-11-30,-1.25,34.75,286.370056,0,0 days,0.0


In [ ]:
# convert grid cells points to shapely points
gdf = gpd.GeoDataFrame(
    df_dewpoint,
    geometry=gpd.points_from_xy(df_dewpoint.longitude, df_dewpoint.latitude),
    crs="EPSG:4326",
)

In [ ]:
gdf = gpd.sjoin(gdf, ug, how="left", predicate="within")
gdf

,time,latitude,longitude,d2m,number,step,surface,geometry,index_right,ADM2_EN,ADM1_EN
0,1981-01-31,4.00,29.75,287.893951,0,0 days,0.0,POINT (29.75 4),NaN,NaN,NaN
1,1981-01-31,4.00,30.00,287.198517,0,0 days,0.0,POINT (30 4),NaN,NaN,NaN
2,1981-01-31,4.00,30.25,286.702942,0,0 days,0.0,POINT (30.25 4),NaN,NaN,NaN
3,1981-01-31,4.00,30.50,285.920441,0,0 days,0.0,POINT (30.5 4),NaN,NaN,NaN
4,1981-01-31,4.00,30.75,285.420624,0,0 days,0.0,POINT (30.75 4),NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
260871,2025-11-30,-1.25,34.00,291.429352,0,0 days,0.0,POINT (34 -1.25),NaN,NaN,NaN
260872,2025-11-30,-1.25,34.25,289.202240,0,0 days,0.0,POINT (34.25 -1.25),NaN,NaN,NaN
260873,2025-11-30,-1.25,34.50,287.637360,0,0 days,0.0,POINT (34.5 -1.25),NaN,NaN,NaN
260874,2025-11-30,-1.25,34.75,286.370056,0,0 days,0.0,POINT (34.75 -1.25),NaN,NaN,NaN


In [ ]:
df_monthly = gdf.groupby(["ADM2_EN", "ADM1_EN", "time"])["d2m"].mean().reset_index()
df_monthly

,ADM2_EN,ADM1_EN,time,d2m
0,Abim,Northern,1981-01-31,281.072662
1,Abim,Northern,1981-02-28,281.732056
2,Abim,Northern,1981-03-31,288.195923
3,Abim,Northern,1981-04-30,291.586578
4,Abim,Northern,1981-05-31,291.480713
...,...,...,...,...
64675,Zombo,Northern,2025-07-31,289.630157
64676,Zombo,Northern,2025-08-31,290.048248
64677,Zombo,Northern,2025-09-30,289.948639
64678,Zombo,Northern,2025-10-31,289.974365


In [ ]:
# rename columns
df_monthly = df_monthly.rename(
    columns={
        "ADM2_EN": "District",
        "ADM1_EN": "Region",
        "time": "Date",
        "d2m": "Dewpoint_Temperature",
    }
)
df_monthly

,District,Region,Date,Dewpoint_Temperature
0,Abim,Northern,1981-01-31,281.072662
1,Abim,Northern,1981-02-28,281.732056
2,Abim,Northern,1981-03-31,288.195923
3,Abim,Northern,1981-04-30,291.586578
4,Abim,Northern,1981-05-31,291.480713
...,...,...,...,...
64675,Zombo,Northern,2025-07-31,289.630157
64676,Zombo,Northern,2025-08-31,290.048248
64677,Zombo,Northern,2025-09-30,289.948639
64678,Zombo,Northern,2025-10-31,289.974365


In [ ]:
# save to csv
df_monthly.to_csv("uganda_monthly_dewpoint_temperature_1981_2025.csv", index=False)